The purpose of this notebook is to document process of preparation dind-black-is-white docker image.

We use source code from https://github.com/abbbe/moby repo, black_is_white branch.

## Install docker

https://docs.docker.com/engine/install/ubuntu/#install-using-the-repository

*NB*: assume sudo does not ask for password

In [4]:
sudo apt-get update
sudo apt-get install -y ca-certificates curl gnupg

Hit:1 http://be.archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://be.archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://be.archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
                        Reading package lists... 0%Reading package lists... 0%Reading package lists... 0%Reading package lists... 3%Reading package lists... 3%Reading package lists... 5%Reading package lists... 5%Reading package lists... 7%Reading package lists... 7%Reading package lists... 7%Reading package lists... 7%Reading package lists... 7%Reading package lists... 7%Reading package lists... 8%Reading package lists... 8%Reading package lists... 40%Reading package lists... 40%Reading package lists... 56%Reading package lists... 56%Reading package lists... 71%Reading package lists... 71%Reading package lists... 72%Reading package lists... 72%Reading package lists... 72%Reading package lists... 72%Reading package lists... 72%Readin

In [5]:
sudo install -m 0755 -d /etc/apt/keyrings
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /etc/apt/keyrings/docker.gpg
sudo chmod a+r /etc/apt/keyrings/docker.gpg

In [6]:
echo \
  "deb [arch="$(dpkg --print-architecture)" signed-by=/etc/apt/keyrings/docker.gpg] https://download.docker.com/linux/ubuntu \
  "$(. /etc/os-release && echo "$VERSION_CODENAME")" stable" | \
  sudo tee /etc/apt/sources.list.d/docker.list > /dev/null

In [7]:
sudo apt-get update

Get:1 https://download.docker.com/linux/ubuntu jammy InRelease [48,9 kB]
Get:2 https://download.docker.com/linux/ubuntu jammy/stable amd64 Packages [21,4 kB]
Hit:3 http://be.archive.ubuntu.com/ubuntu jammy InRelease                      
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease               
Hit:5 http://be.archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://be.archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 70,3 kB in 1s (109 kB/s)
Reading package lists... 0%Reading package lists... 0%Reading package lists... 0%Reading package lists... 3%Reading package lists... 3%Reading package lists... 5%Reading package lists... 5%Reading package lists... 7%Reading package lists... 7%Reading package lists... 7%Reading package lists... 7%Reading package lists... 7%Reading package lists... 7%Reading package lists... 8%Reading package lists... 8%Reading package lists... 39%Reading package lists... 39%Reading package lists... 56%Reading package lists... 5

In [9]:
sudo apt-get install -y docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin

Reading package lists... 0%Reading package lists... 100%Reading package lists... Done
Building dependency tree... 0%Building dependency tree... 0%Building dependency tree... 50%Building dependency tree... 50%Building dependency tree... Done
Reading state information... 0% Reading state information... 0%Reading state information... Done
The following additional packages will be installed:
  docker-ce-rootless-extras libslirp0 pigz slirp4netns
Suggested packages:
  aufs-tools cgroupfs-mount | cgroup-lite
The following NEW packages will be installed:
  containerd.io docker-buildx-plugin docker-ce docker-ce-cli
  docker-ce-rootless-extras docker-compose-plugin libslirp0 pigz slirp4netns
0 upgraded, 9 newly installed, 0 to remove and 4 not upgraded.
Need to get 114 MB of archives.
After this operation, 414 MB of additional disk space will be used.
Get:1 https://download.docker.com/linux/ubuntu jammy/stable amd64 containerd.io amd64 1.6.22-1 [28,3 MB]
Get:2 http://be.archive.ubuntu.com/ubunt

In [10]:
sudo docker run hello-world

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world

Digest: sha256:dcba6daec718f547568c562956fa47e1b03673dd010fe6ee58ca806767031d1c
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https

In [13]:
sudo usermod -aG docker $USER

*NB*: restart jupyter from a fresh shell session to apply

## Moby

In [46]:
WD=$(mktemp -d)

### fetch source code

In [ ]:
#mkdir -p $GOPATH/src/github.com/moby
#cd $GOPATH/src/github.com/moby
#[ -d moby ] ||
git clone https://github.com/moby/moby.git

In [18]:
cd moby

### install dependencies

In [ ]:
sudo snap install go --classic
go get -d ./...
go install github.com/LK4D4/vndr@latest

*NB*: vndr executable should end up in $PATH for instance in ~/.local/bin

### test unmodified code

In [ ]:
make binary

In [ ]:
make test

In [ ]:
TESTDIRS=./container/stream make test-unit

### patch and test

In [ ]:
TEST_SKIP_INTEGRATION_CLI=1 make test-integration TESTFLAGS="-test.run TestExecBlackIsWhite"

The command above takes a long time to run. Also, it runs tests from various directories and you do no necessarily see if your tests has failed.

```
...
Running /go/src/github.com/docker/docker/integration/container (amd64.integration.container) flags=-test.v -test.timeout=5m -test.run TestExecBlackIsWhite 
INFO: Testing against a local daemon
=== RUN   TestExecBlackIsWhite
--- PASS: TestExecBlackIsWhite (0.65s)
PASS

DONE 1 tests in 0.677s
...
```

To quickly run our test only:

```
$ make BIND_DIR=. shell
# ./bundles/dynbinary-daemon/dockerd &
# go test -v ./integration/container -run TestExecBlackIsWhite
Loaded image: busybox:latest
Loaded image: busybox:glibc
Loaded image: debian:bullseye-slim
Loaded image: hello-world:latest
Loaded image: arm32v7/hello-world:latest
INFO: Testing against a local daemon
=== RUN   TestExecBlackIsWhite
time="2023-08-27T18:21:36.197724895Z" level=info msg="loading plugin \"io.containerd.internal.v1.shutdown\"..." runtime=io.containerd.runc.v2 type=io.containerd.internal.v1
time="2023-08-27T18:21:36.197773274Z" level=info msg="loading plugin \"io.containerd.ttrpc.v1.pause\"..." runtime=io.containerd.runc.v2 type=io.containerd.ttrpc.v1
time="2023-08-27T18:21:36.197785044Z" level=info msg="loading plugin \"io.containerd.event.v1.publisher\"..." runtime=io.containerd.runc.v2 type=io.containerd.event.v1
time="2023-08-27T18:21:36.197792335Z" level=info msg="loading plugin \"io.containerd.ttrpc.v1.task\"..." runtime=io.containerd.runc.v2 type=io.containerd.ttrpc.v1
INFO[2023-08-27T18:21:36.498614412Z] shim disconnected                             id=a2852cf866651e8283a6808fe3058e4c1a35b2d26097a800877cbade99bebf5d namespace=moby
WARN[2023-08-27T18:21:36.498685085Z] cleaning up after shim disconnected           id=a2852cf866651e8283a6808fe3058e4c1a35b2d26097a800877cbade99bebf5d namespace=moby
INFO[2023-08-27T18:21:36.498752438Z] ignoring event                                container=a2852cf866651e8283a6808fe3058e4c1a35b2d26097a800877cbade99bebf5d module=libcontainerd namespace=moby topic=/tasks/delete type="*events.TaskDelete"
INFO[2023-08-27T18:21:36.498787843Z] cleaning up dead shim                         namespace=moby
--- PASS: TestExecBlackIsWhite (0.69s)
PASS
ok  	github.com/docker/docker/integration/container	1.919s
```

### build and leave moby/

In [ ]:
make binary
cd ..

## Build patched docker:dind image

#### build

In [48]:
cp moby/bundles/binary/dockerd $WD/

cat << END > $WD/Dockerfile.dind-black-is-white
# Use the official docker:dind as a base
FROM docker:dind

COPY ./dockerd /usr/local/bin/dockerd
END

In [51]:
docker build -t dind-black-is-white:dind -f $WD/Dockerfile.dind-black-is-white $WD

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (7/7)                                          docker:default
 => [internal] load build definition from Dockerfile.dind-black-is-white   0.0s
 => => transferring dockerfile: 154B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/docker:dind             0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 31B                                           0.0s
 => [1/2] FROM docker.io/library/docker:dind                               0.0s
 => CACHED [2/2] COPY ./dockerd /usr/local/bin/dockerd                     0.0s
 => exporting to image                                                     0.0s
 => => exporting layers                 

In [52]:
docker kill dind-black-is-white
docker run --rm --name dind-black-is-white -d --privileged -it dind-black-is-white:dind

Error response from daemon: Cannot kill container: dind-black-is-white: No such container: dind-black-is-white
df3d8a8ee2a9dc16fcd8b4ec0f3a87c839caf674a14633eb6095be53c4b2979e


In [53]:
docker exec -it dind-black-is-white \
    docker --version

Docker version 24.0.5, build ced0996


In [54]:
docker exec -it docker-black-is-white \
    docker run busybox echo "{black}"

{white}
